In [ ]:
# This file has no websockets and just serves basic html text file
# Use as basis for building up more complex things (eg with loops running sensors)

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [ ]:
# perhaps we can precompile the file below

In [2]:
%sendtofile --quiet --source ../async_web_serve/webserve_funcs.py
%sendtofile --quiet --source ../basicsockets/syslog.py
%comment %sendtofile --quiet --source ../async_web_serve/websocket_funcs.py

Sent 190 lines (6050 bytes) to webserve_funcs.py.
Sent 34 lines (705 bytes) to syslog.py.
%sendtofile --quiet --source ../async_web_serve/websocket_funcs.py

In [3]:
%sendtofile --source main.py


Sent 219 lines (6631 bytes) to main.py.


In [65]:
%disconnect


Closing serial Serial<id=0x7fe80a52b438, open=True>(port='/dev/ttyUSB3', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [58]:
%rebootdevice

[leftinbuffer] ['ets Jun  8 2016 00:22:57']
[leftinbuffer] ['']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:4332']
[leftinbuffer] ['load:0x40078000,len:0']
[leftinbuffer] ['load:0x40078000,len:10992']
[leftinbuffer] ['entry 0x4007a6c4']
[leftinbuffer] ['\x1bW (64) rtc_clk: Potentially bogus XTAL frequency: 35 MHz, guessing 40 MHz\x1b']
[leftinbuffer] ['OSError: [Errno 2] ENOENT']
[leftinbuffer] ['MicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32']
[leftinbuffer] ['Type "help()" for more information.']
[leftinbuffer] ['>>> ']
repl is in normal command mode
[\r\x03\x03] b'\r\n>>> \r\n>>> \r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for mor

In [7]:
%%writefile main.py

import time
time.sleep(1.0)

#from connectwifi import ipnumber
from connecthotspot import ipnumber



Writing main.py



In [8]:
%%writefile --append main.py

from webserve_funcs import servestaticfile
from webserve_funcs import servefilelist
from webserve_funcs import readhttpheaders
from webserve_funcs import servemessage

#from vl6180_funcs import init1, init2, set_reg8, get_reg8, distmm
#from ms5611_funcs import setupms5611, aMS5611read


Appending to main.py



In [9]:
%%writefile --append main.py

# Optional file writing of humidity

import uasyncio
import time, ustruct
from si7021_funcs import setupSI7021, SI7021humiditytemp, DewpointTemperature, aSI7021humiditytemp

nfiles, nvals, nsleep = 10, 10000, 500
#nfiles, nvals, nsleep = 4, 50, 20
bstop = False
async def humiditything():
    setupSI7021()
    for i in range(nfiles):
        print("starting", i)
        t0 = time.ticks_ms()
        fout = open("dat/ht%d.bin"%int(t0//1000), "wb")
        for j in range(nvals):
            await uasyncio.sleep_ms(500)
            h, temp = await aSI7021humiditytemp(uasyncio)
            t = time.ticks_ms()
            fout.write(ustruct.pack("<HH", t-t0, int(temp*100)))
            t0 = t
            if bstop:
                break
            if (j%20) == 19:
                fout.flush()
        fout.close()
        if bstop:
            break
    print("Done humidity thing")

import os
def cleardatdir():
    for l in os.listdir("dat"):
        os.remove("dat/"+l)

Appending to main.py



In [10]:
%%writefile --append main.py

import os, uasyncio
async def handleconnection(reader, writer):
    receivedrequest = await readhttpheaders(reader)
    rpath = receivedrequest.get("GET")
    print(receivedrequest)
    
    bcallaclose = True
    if rpath == "":
        await servefilelist(writer)
    elif rpath == "go":
        cleardatdir()
        loop.create_task(humiditything())
        await servemessage(writer, "got humid")
    elif rpath:
        await servestaticfile(writer, rpath)
        
    if bcallaclose:
        await writer.aclose()


Appending to main.py



In [11]:
%%writefile --append main.py

import uasyncio
uasyncio.set_debug(0)

# enables rewriting handleconnection function without restarting
def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))
    
port = 80
loop = uasyncio.get_event_loop()
print("Will running on http://%s:%s/" % (ipnumber, port))
loop.create_task(uasyncio.start_server(handleconnection_indirect, ipnumber, port))


Appending to main.py



In [12]:
%%writefile --append main.py
loop.run_forever()


Appending to main.py



In [15]:
%serialconnect

Connecting to Serial /dev/ttyUSB2 baud=115200 
ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4332
load:0x40078000,len:0
load:0x40078000,len:10992
entry 0x4007a6c4
W (64) rtc_clk: Potentially bogus XTAL frequency: 35 MHz, guessing 40 MHz
I (543) network: event 13
Creating access point ESP_0e3acd
Device has ipnumber 192.168.4.1


*** Sending Ctrl-C

Traceback (most recent call last):
  File "main.py", line 67, in <module>
  File "/lib/uasyncio/__init__.py", line 13, in set_debug
KeyboardInterrupt: 
MicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' 

In [16]:
%sendtofile --source main.py

Sent 82 lines (2380 bytes) to main.py.


In [57]:
cleardatdir()

In [69]:
cleardatdir()
loop.create_task(humiditything())


In [70]:
loop.run_forever()


starting 0
.[] serving file list
.[] serving file list
[] serving file list
.[] serving file list
..starting 1
.....starting 2
......starting 3
.....Done humidity thing
..............................................................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/lib/uasyncio/core.py", line 77, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


object <module 'uasyncio' from '/lib/uasyncio/__init__.py'> is of type module
  _socket -- <module 'usocket'>
  StopLoop -- <class 'StopLoop'>
  EventLoop -- <class 'EventLoop'>
  start_server -- <generator>
  IORead -- <class 'IORead'>
  sleep -- <generator>
  IOWriteDone -- <class 'IOWriteDone'>
  ensure_future -- <function ensure_future at 0x3ffb77e0>
  set_debug -- <function set_debug at 0x3ffb6600>
  sleep_ms -- <SleepMs object at 3ffb79c0>
  utimeq -- <module 'utimeq'>
  __file__ -- /lib/uasyncio/__init__.py
  SleepMs -- <class 'SleepMs'>
  IOReadDone -- <class 'IOReadDone'>
  coroutine -- <function coroutine at 0x3ffb79e0>
  core -- <module 'uasyncio.core' from '/lib/uasyncio/core.py'>
  type_gen -- <class 'generator'>
  SysCall1 -- <class 'SysCall1'>
  SysCall -- <class 'SysCall'>
  select -- <module 'uselect'>
  DEBUG -- 0
  __name__ -- uasyncio
  __path__ -- /lib/uasyncio
  StreamWriter -- <class 'StreamWriter'>
  uasyncio -- <module 'uasyncio' from '/lib/uasyncio/__init__.py

In [34]:
import time
try:
    1/0
    
except Exception as e:
    print([e])
    sys.print_exception(e)
    print(instanceOf(Exception))
#raise
import sys


Traceback (most recent call last):
  File "<stdin>", line 8
SyntaxError: invalid syntax


In [39]:
fout = open("test.txt", "w")
sys.print_exception("ggg", fout)
fout.close()

In [40]:
print([open("test.txt").read()])

["'ggg'\n"]
